In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import linear_model

# visualization
import matplotlib.pyplot as plt
import seaborn as sns    #是在matplotlib的基础上进行了更高级的API封装，从而使得作图更加容易，在大多数情况下使用seaborn就能做出很具有吸引力的图
%matplotlib inline

from sklearn.model_selection import train_test_split # utils
from sklearn.metrics import mean_absolute_error # eval metric
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import  mean_squared_error 
from sklearn.metrics import median_absolute_error 
from sklearn.metrics import r2_score 

# data processing
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

# from sklearn.linear_model import ElasticNet # machine learning

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.path.listdir("../input"))

# Read data

In [20]:
df = pd.read_csv('X_train.csv')
y = pd.read_csv('y_train.csv')['PRP']
df_test = pd.read_csv('X_test.csv')
df.head()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX
0,125,256,6000,256,16,128
1,29,8000,32000,32,8,32
2,29,8000,32000,32,8,32
3,29,8000,32000,32,8,32
4,26,8000,32000,64,8,32


# Split data with percentages

In [21]:
# split the data
X, y = df.values, y.values
X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.1,random_state = 1)

# data preprocessing using sklearn Pipeline
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, interaction_only=True)), # multiply features together
    ('scale', StandardScaler()), # scale data
])

# fit and apply transform
X_train = pipeline.fit_transform(X_train)
# # transform the validation set
X_val = pipeline.transform(X_val)

# print(X_train)
# print( X_val)

# from sklearn import preprocessing
# scaler = StandardScaler()
# # fit and apply transform
# X_train = scaler.fit_transform(X_train)
# # transform the validation set
# X_val = scaler.transform(X_val)
# print('train shape:', X_train.shape, 'validation shape:', X_val.shape)


# Regression model- Bayesian ARD regression

In [22]:
#Fit the weights of a regression model, using an ARD prior. 
#The weights of the regression model are assumed to be in Gaussian distributions. 
clf = linear_model.ARDRegression()
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_val)
y_pred[y_pred < 0] = 0
loss = mean_absolute_error(y_val, y_pred)
evs = explained_variance_score(y_val, y_pred)
#Best possible score is 1.0, lower values are worse.
mse = mean_squared_error(y_val, y_pred)
#(the best value is 0.0)
mae = median_absolute_error(y_val, y_pred)
#(the best value is 0.0).
r2 = r2_score(y_val, y_pred)
#Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 
print('Model MAE:', loss)
print('Explained Variance Score:', evs)
print('Mean Squared Error:', mse)
print('Median_Absolute_Error:', mae)
print('r2_score:', r2)

Model MAE: 13.11400519858643
Explained Variance Score: 0.9456972843819389
Mean Squared Error: 289.26243493178566
Median_Absolute_Error: 8.529083009323386
r2_score: 0.9423240930912652


#  Prediction- Bayesian ARD regression

In [23]:
# refit and predict submission data
X_train = pipeline.fit_transform(X)
X_test = pipeline.transform(df_test.values)
clf.fit(X_train, y)
y_pred = clf.predict(X_test)
y_pred[y_pred < 0] = 0

df_sub = pd.DataFrame({'Id': np.arange(y_pred.size), 'PRP': y_pred})
df_sub.to_csv('submission_Bayesian regression.csv', index=False)